# Activating Function Analysis

Analyze activation of neurite structured by DBS-imposed electric field

In [ ]:
import cPickle as pickle
import re, os

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Figure dimensions
fig_aspect_ratio = 8.0 / 3.0
page_width = 10
fig_width = 0.8 * page_width
fig_height = fig_width / fig_aspect_ratio
ax_width = 0.7 * page_width
ax_height = ax_width / fig_aspect_ratio

# Style of figures (default colors etc.): see https://matplotlib.org/gallery/style_sheets/style_sheets_reference.html
plt.style.use('default')

# Consistent size in backend 'notebook' (100 dpi) and 'inline' (72 dpi)
def plt_setrc():
    matplotlib.rcParams['savefig.dpi'] = 100
    matplotlib.rcParams['figure.dpi'] = 100

In [ ]:
%matplotlib notebook
plt_setrc()

# Import Data

In [ ]:
# SETPARAM: path to pickle files with population data
outputs = "/home/luye/cloudstore_m/simdata/LuNetDBS/LuNetDBS_2019.08.28_13.51.43_job-ActFun_axons-full-V4_gNMDA-low_template"

file_filter = lambda f: f.endswith('.pkl') and f.startswith('comp-locs')

filenames = os.listdir(outputs)
pkl_files = [os.path.join(outputs, f) for f in filenames if file_filter(f)]

# Structure
# 'STN' : {
#   'comp_act' : { 'somatic': list[float], 'basal': list[float], 'axonal': list[float]}
#   'comp_dists' : { 'somatic': list[float], 'basal': list[float], 'axonal': list[float]}
# }
#     
all_pop_data = {}
all_cell_offsets = {}

for pkl_fname in pkl_files:
    matches = re.search(r'comp-locs_([\w\.]+)_', pkl_fname)
    pop_label = matches.groups()[0]
    
    with open(pkl_fname, 'rb') as f:
        data = pickle.load(f)

    # Pooled data
    all_pop_data.setdefault(pop_label, {})['comp_act'] = all_cells_act = data['comp_act'][0]
    all_pop_data.setdefault(pop_label, {})['comp_dists'] = all_cells_dists = data['comp_dists'][0]
    
    # Store offset to data for each cell
    all_cell_offsets[pop_label] = {k: [0, len(vals)] for k, vals in all_cells_act.iteritems()}
    
    for i in range(1, len(data['comp_act'])):
        cell_act = data['comp_act'][i]
        cell_dists = data['comp_dists'][i]
        
        
        # Append this cell's data to population data
        for k in cell_act.keys():
            all_cells_act[k].extend(cell_act[k])
            all_cell_offsets[pop_label][k].append(all_cell_offsets[pop_label][k][-1] +  len(cell_act[k]))
            
        for k in cell_dists.keys():
            all_cells_dists[k].extend(cell_dists[k])

    # Convert to numpy array
    for k in all_cells_act.keys():
        all_cells_act[k] = np.array(all_cells_act[k])
    for k in all_cells_dists.keys():
        all_cells_dists[k] = np.array(all_cells_dists[k])

# Scatter plots

In [ ]:
def plot_actfun_scatter(pop_label, multiple_figs=False, cells='all'):
    
    if cells == 'all':
        regions_acts = all_pop_data[pop_label]['comp_act']
        regions_dists = all_pop_data[pop_label]['comp_dists']
    else:
        raise NotImplementedError()
#         regions_slices = {region: slice(all_cell_offsets[pop_label][region][cells]) }
#         regions_acts = all_pop_data[pop_label]['comp_act']
#         regions_dists = all_pop_data[pop_label]['comp_dists']
    
    if not multiple_figs:
        fig, ax = plt.subplots(figsize=(fig_width, ax_height))
        
    def decorate_fig(fig, ax, region=None):
        ax.legend()
        ax.grid()
        ax.set_xlabel('dist. from soma (um)')
        ax.set_ylabel('activating func. (mV/ms)')
        
        title = "Activating function"
        if region:
            title += " ({} compartments)".format(region)
        ax.set_title(title)
        fig.subplots_adjust(bottom=0.15) # prevent clipping of xlabel

    # Plot distance vs activating function
    for region in regions_acts.keys():
        if multiple_figs:
            fig, ax = plt.subplots(figsize=(fig_width, ax_height))
        
        ax.plot(regions_dists[region], regions_acts[region],
                '.', label=region, markersize=3)

        if multiple_figs:
            decorate_fig(fig, ax, region=region)
        
    if not multiple_figs:
        decorate_fig(fig, ax)

## STN

In [ ]:
plot_actfun_scatter('STN', multiple_figs=True)

## GPE

In [ ]:
plot_actfun_scatter('GPE.proto', multiple_figs=True)

# Boxplots

In [ ]:
def plot_actfun_boxplot(pop_label):

    regions = 'somatic', 'basal', 'axonal'
    fig, axes = plt.subplots(1, len(regions), figsize=(page_width, 1.5*ax_height))

    for i, region in enumerate(regions):
        ax = axes[i]
        ax.set_title("Act. func. ({})".format(region))

        x = all_pop_data[pop_label]['comp_act'][region]
        bp = ax.boxplot(x, 0, 'g+')
        ax.grid(True, which='major', axis='y')
        ax.set_xticks([])

        print("\nStatistics ({}):".format(region))
        print("- std : {:.3f}".format(np.std(x)))

## STN

In [ ]:
plot_actfun_boxplot('STN')

## GPE

In [ ]:
plot_actfun_boxplot('GPE.proto')